# 第5章: 係り受け解析

夏目漱石の小説『吾輩は猫である』の文章（[neko.txt](https://nlp100.github.io/data/neko.txt)）をCaboChaを使って係り受け解析し，その結果をneko.txt.cabochaというファイルに保存せよ．このファイルを用いて，以下の問に対応するプログラムを実装せよ．

# 40. 係り受け解析結果の読み込み（形態素）

形態素を表すクラスMorphを実装せよ．このクラスは表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をメンバ変数に持つこととする．さらに，CaboChaの解析結果（neko.txt.cabocha）を読み込み，各文をMorphオブジェクトのリストとして表現し，3文目の形態素列を表示せよ．

In [1]:
neko_cabocha_file = '/Users/seiji/言語処理100本ノック/5係り受け解析/neko.txt.cabocha'



class Morph:
    """
    1つの形態素を表すクラス
    """
    
    def __init__(self, surface, base, pos, pos1):
        """
        メンバ変数として表層形（surface）, 基本形（base）, 品詞（pos）,　品詞細分類１（pos1）を持つ
        """
        self.surface = surface
        self.base = base
        self.pos = pos
        self.pos1 = pos1
    
    
def make_morph_list(filename):
    """
    係り受け解析済みのファイルを読み込み、各文をMorphオブジェクトのリストとして表現する
    """
    
    sentences = []
    sentence = []
    with open(filename, encoding='utf-8') as input_file:
        for line in input_file:
            if line[0] == '*':
                next
            if '\t' in line:
                item = line.strip().split('\t')
                try:
                    surf = item[0]
                    items = item[1].split(',')
                except IndexError:
                    next
                if not item == ['記号,空白,*,*,*,*,\u3000,\u3000,']: #　空白の箇所の処理、\u3000は空白　
                    sentence.append(Morph(surf, items[6], items[0], items[1]))
                    
            
                    if '句点' in item[1]:
                        sentences.append(sentence)
                        sentence = []
                    
    return sentences

In [2]:
morphed_sentences = make_morph_list(neko_cabocha_file)

In [3]:
for item in morphed_sentences[2]:
    print ('surface: {}, base: {}, pos: {}, pos1: {}'.format(item.surface, item.base, item.pos, item.pos1))

surface: どこ, base: どこ, pos: 名詞, pos1: 代名詞
surface: で, base: で, pos: 助詞, pos1: 格助詞
surface: 生れ, base: 生れる, pos: 動詞, pos1: 自立
surface: た, base: た, pos: 助動詞, pos1: *
surface: かとん, base: 火遁, pos: 名詞, pos1: 一般
surface: と, base: と, pos: 助詞, pos1: 格助詞
surface: 見当, base: 見当, pos: 名詞, pos1: サ変接続
surface: が, base: が, pos: 助詞, pos1: 格助詞
surface: つか, base: つく, pos: 動詞, pos1: 自立
surface: ぬ, base: ぬ, pos: 助動詞, pos1: *
surface: 。, base: 。, pos: 記号, pos1: 句点


# 41. 係り受け解析結果の読み込み（文節・係り受け）

40に加えて，文節を表すクラスChunkを実装せよ．このクラスは形態素（Morphオブジェクト）のリスト（morphs），係り先文節インデックス番号（dst），係り元文節インデックス番号のリスト（srcs）をメンバ変数に持つこととする．さらに，入力テキストのCaboChaの解析結果を読み込み，１文をChunkオブジェクトのリストとして表現し，8文目の文節の文字列と係り先を表示せよ．第5章の残りの問題では，ここで作ったプログラムを活用せよ．

In [4]:
import re

neko_cabocha_file = '/Users/seiji/言語処理100本ノック/5係り受け解析/neko.txt.cabocha'

class Morph(object):
    def __init__(self, line):
        self.surface, rest = line.split('\t')
        attr = rest.split(',')
        self.base = attr[6]
        self.pos = attr[0]
        self.pos1 = attr[1]
        
    def __str__(self):
        return 'surface:{}/base:{}/pos:{}/pos1:{}\n'.format(
            self.surface, self.base, self.pos, self.pos1
        )

class Chunk(object):
    def __init__(self, line):
        self.morphs = []
        self.dst = int(line.split(' ')[2].rstrip('D'))
        self.srcs = []

def read_cabocha_file(filename):
    sentence = []
    chunk = None
    with open(filename, mode='rt', encoding='utf-8') as input_file:
        for line in input_file:
            line = line.rstrip('\n')
            if line == 'EOS':
                #sentenceのsrcを設定していく
                for index, chunk in enumerate(sentence):
                    # 係り先が -1の時は係り先がないから飛ばす
                    if chunk.dst != -1:
                        sentence[chunk.dst].srcs.append(index)
                                
                yield sentence
                sentence = []
            elif re.match(r'^\*[^\t]+$', line):
                #'*'が形態素になってる可能性もあるので、
                #タブを含まない行の行頭が '*'　な場合だけ読み飛ばす
                chunk = Chunk(line)
                sentence.append(chunk)
            else:
                chunk.morphs.append(Morph(line))


In [5]:
if __name__ == '__main__':
    sentences = list(read_cabocha_file(neko_cabocha_file))
    for i, chunk in enumerate(sentences[7]):
        print('* ', i)
        print('srcs: {}, dst: {}'.format(chunk.srcs, chunk.dst))
        print('morphs: {}'.format(''.join([str(m) for m in chunk.morphs])))
    import doctest
    doctest.testmod()

*  0
srcs: [], dst: 5
morphs: surface:吾輩/base:吾輩/pos:名詞/pos1:代名詞
surface:は/base:は/pos:助詞/pos1:係助詞

*  1
srcs: [], dst: 2
morphs: surface:ここ/base:ここ/pos:名詞/pos1:代名詞
surface:で/base:で/pos:助詞/pos1:格助詞

*  2
srcs: [1], dst: 3
morphs: surface:始め/base:始める/pos:動詞/pos1:自立
surface:て/base:て/pos:助詞/pos1:接続助詞

*  3
srcs: [2], dst: 4
morphs: surface:人間/base:人間/pos:名詞/pos1:一般
surface:という/base:という/pos:助詞/pos1:格助詞

*  4
srcs: [3], dst: 5
morphs: surface:もの/base:もの/pos:名詞/pos1:非自立
surface:を/base:を/pos:助詞/pos1:格助詞

*  5
srcs: [0, 4], dst: -1
morphs: surface:見/base:見る/pos:動詞/pos1:自立
surface:た/base:た/pos:助動詞/pos1:*
surface:。/base:。/pos:記号/pos1:句点



# 42. 係り元と係り先の文節の表示

係り元の文節と係り先の文節のテキストをタブ区切り形式ですべて抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [6]:
chunk_src_and_dst_list = []
paired_list = []
paired_sentences = []

# 一文ごとに処理
for sentence in sentences:
    if len(sentence) > 1:

        for chunk in sentence:
            # 係り元の文節と係り先の文節を抽出、
            # dst: -1は係り先がないから飛ばす。
            if chunk.dst == -1:
                pass
            else:
                # 係り元
                chunk_src = ''.join([m.surface for m in chunk.morphs])
                # 係り先
                chunk_dst = ''.join([m.surface for m in sentence[chunk.dst].morphs])

                #句読点を出力しないようにする
                chunk_src_and_dst = chunk_src + '\t' + chunk_dst
                chunk_src_and_dst = re.sub(r'[、。\u3000]', '', chunk_src_and_dst)

                chunk_src_and_dst_list.append(chunk_src_and_dst)
                paired_list.append(chunk_src_and_dst)
        
        paired_sentences.append(chunk_src_and_dst_list) 
        chunk_src_and_dst_list = []

In [7]:
print(paired_sentences[3])
print(paired_sentences[3][0])

['吾輩は\t見た', 'ここで\t始めて', '始めて\t人間という', '人間という\tものを', 'ものを\t見た']
吾輩は	見た


* 「じめじめした所で」を文節に分けると、[じめじめした, 所で]であるが、出力結果は異なっている。これは係り受け解析器の精度の問題であろう。

# 43. 名詞を含む文節が動詞を含む文節に係るものを抽出

名詞を含む文節が，動詞を含む文節に係るとき，これらをタブ区切り形式で抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [8]:
chunk_noun_to_verb_list = []

# 一文ごとに処理
for sentence in sentences:
    
    for chunk in sentence:
        # 係り元の文節と係り先の文節を抽出、
        # dst: -1は係り先がないから飛ばす。
        if chunk.dst == -1:
            pass
        else:
            
            if ('名詞' in [m.pos for m in chunk.morphs]) & ('動詞' in [m.pos for m in sentence[chunk.dst].morphs]):
                # 名詞
                chunk_noun = ''.join([m.surface for m in chunk.morphs])
                # 動詞
                chunk_verb = ''.join([m.surface for m in sentence[chunk.dst].morphs])
                
                # タブ区切り、句読点を出力しないようにする
                chunk_noun_to_verb = chunk_noun + '\t' + chunk_verb
                chunk_noun_to_verb = re.sub(r'[、。\u3000]', '', chunk_noun_to_verb)
                
                chunk_noun_to_verb_list.append(chunk_noun_to_verb)

In [9]:
print(chunk_noun_to_verb_list[:10])
print(len(chunk_noun_to_verb_list))

['どこで\t生れた', 'かとんと\tつかぬ', '見当が\tつかぬ', 'した所で\t泣いて', 'いた事だけは\t記憶している', '吾輩は\t見た', 'ここで\t始めて', 'ものを\t見た', 'あとで\t聞くと', '我々を\t捕えて']
28731


# 44. 係り受け木の可視化

与えられた文の係り受け木を有向グラフとして可視化せよ．可視化には，係り受け木を[DOT言語](http://ja.wikipedia.org/wiki/DOT言語)に変換し，[Graphviz](http://www.graphviz.org/)を用いるとよい．また，Pythonから有向グラフを直接的に可視化するには，[pydot](https://code.google.com/p/pydot/)を使うとよい．

In [10]:
import pydotplus as pdp

def make_dot_edge(sentence):
    body_edge = []
    
    for chunk_pair in sentence:
        src = chunk_pair.split('\t')[0]
        dst = chunk_pair.split('\t')[1]
        
        body_edge.append('{}->{}; '.format(src, dst))
        
    return body_edge

def sentence_to_dot(idx, sentence):
    head = "digraph sentence{}".format(idx)
    #グラフを左から右に可視化
    body_graph = "{ graph [rankdir = LR];"
    body_edge = make_dot_edge(sentence)
    
    return head + body_graph + ''.join(body_edge) + '}'
    

def sentences_to_dots(sentences):
    _dots = []
    for idx, sentence in enumerate(sentences):
        _dots.append(sentence_to_dot(idx, sentence))
    
    return _dots

def save_graph(dot, filename):
    g = pdp.graph_from_dot_data(dot)
    g.write_jpeg(filename, prog='dot')
    
dots = sentences_to_dots(paired_sentences)
for idx in range(101, 104):
    save_graph(dots[idx], 'graph{}.jpg'.format(idx))